In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

%matplotlib inline

In [2]:
df = pd.read_csv('covid.csv')

In [3]:
df.columns

Index(['id', 'sex', 'patient_type', 'entry_date', 'date_symptoms', 'date_died',
       'intubed', 'pneumonia', 'age', 'pregnancy', 'diabetes', 'copd',
       'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular',
       'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid',
       'covid_res', 'icu'],
      dtype='object')

In [4]:
df.corr()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
sex,1.000000,0.097025,-0.097029,-0.054758,0.036709,0.994293,0.008153,0.009339,0.010119,0.008096,0.009019,0.010155,0.008344,0.010199,0.008577,0.004980,-0.001791,-0.047575,-0.097024
patient_type,0.097025,1.000000,-0.999319,-0.420400,0.325059,0.095472,0.002315,0.013227,0.016670,0.016359,0.000274,0.028232,0.016446,0.014967,0.013240,0.016603,0.228929,-0.135931,-0.999314
intubed,-0.097029,-0.999319,1.000000,0.421256,-0.324869,-0.095481,-0.002456,-0.013363,-0.016807,-0.016424,-0.000419,-0.028389,-0.016587,-0.015101,-0.013384,-0.016738,-0.228811,0.135738,0.999989
pneumonia,-0.054758,-0.420400,0.421256,1.000000,-0.183492,-0.054031,-0.002875,-0.008592,-0.010370,-0.010979,-0.001568,-0.019541,-0.010392,-0.010070,-0.008296,-0.009996,-0.074756,0.093832,0.421182
age,0.036709,0.325059,-0.324869,-0.183492,1.000000,0.036239,0.000556,0.014440,0.018668,0.018625,-0.008617,0.018627,0.015049,0.012068,0.016643,0.017531,0.099339,-0.102643,-0.324791
pregnancy,0.994293,0.095472,-0.095481,-0.054031,0.036239,1.000000,0.016239,0.018035,0.018931,0.016412,0.017474,0.017392,0.016854,0.018692,0.016970,0.013060,-0.003185,-0.047589,-0.095474
diabetes,0.008153,0.002315,-0.002456,-0.002875,0.000556,0.016239,1.000000,0.838368,0.843817,0.794140,0.845727,0.688245,0.821716,0.765574,0.817052,0.777978,0.002895,0.005156,-0.002429
copd,0.009339,0.013227,-0.013363,-0.008592,0.014440,0.018035,0.838368,1.000000,0.922057,0.866870,0.872321,0.748066,0.893562,0.827598,0.889774,0.844021,0.010322,0.000882,-0.013308
asthma,0.010119,0.016670,-0.016807,-0.010370,0.018668,0.018931,0.843817,0.922057,1.000000,0.886307,0.883549,0.756712,0.906039,0.839128,0.901793,0.855136,0.010714,0.000276,-0.016759
inmsupr,0.008096,0.016359,-0.016424,-0.010979,0.018625,0.016412,0.794140,0.866870,0.886307,1.000000,0.844534,0.800965,0.872862,0.804051,0.866491,0.821285,0.010554,-0.000914,-0.016299


In [15]:
def toBool(value):
    if value in'9999-99-99':
        return 0
    else:
        return 1

In [23]:
dates = df['date_died'].values
new_dates = np.array(list(map(toBool, dates)))
new_dates

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
df['died'] = new_dates

In [29]:
df = df.drop(columns=['id', 'entry_date', 'date_symptoms', 'date_died'])

In [30]:
df.head()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu,died
0,2,1,97,2,27,97,2,2,2,2,2,2,2,2,2,2,2,1,97,0
1,2,1,97,2,24,97,2,2,2,2,2,2,2,2,2,2,99,1,97,0
2,1,2,2,2,54,2,2,2,2,2,2,2,2,1,2,2,99,1,2,0
3,2,2,2,1,30,97,2,2,2,2,2,2,2,2,2,2,99,1,2,0
4,1,2,2,2,60,2,1,2,2,2,1,2,1,2,2,2,99,1,2,1


In [35]:
df.isnull().sum()

sex                    0
patient_type           0
intubed                0
pneumonia              0
age                    0
pregnancy              0
diabetes               0
copd                   0
asthma                 0
inmsupr                0
hypertension           0
other_disease          0
cardiovascular         0
obesity                0
renal_chronic          0
tobacco                0
contact_other_covid    0
covid_res              0
icu                    0
died                   0
dtype: int64